# View Models

This notebook is used for visualizing `.pth` file structures.

In [8]:
import torch
from collections import OrderedDict

The main function we use to inspect a `.pth` file.

In [9]:
def inspect_pth_file(pth_path, max_tensor_elements=5, max_optimizer_params=3):
    print(f"🔍 Inspecting file: {pth_path}\n")
    data = torch.load(pth_path, map_location='cpu')

    if not isinstance(data, dict):
        print(f"[ROOT] Type: {type(data)} (not a dict)")
        return

    print("📁 Top-level Keys and Types:")
    for key in data:
        print(f" - {key}: {type(data[key])}")
    print()

    for key, value in data.items():
        print(f"=== [{key}] ===")
        if isinstance(value, OrderedDict):
            print(f"➡️  Detected state_dict with {len(value)} parameters.")
            for name, tensor in value.items():
                print(f"  - {name:50} | shape: {tuple(tensor.shape)} | dtype: {tensor.dtype}")
        elif isinstance(value, dict) and 'state' in value and 'param_groups' in value:
            print("➡️  Detected optimizer state_dict.")
            state = value['state']
            print(f"  Contains {len(state)} parameter entries.")
            for i, (param_id, param_state) in enumerate(state.items()):
                if i >= max_optimizer_params:
                    print("  ... (truncated)")
                    break
                print(f"  ▶️  Param ID: {param_id}")
                for subkey, subval in param_state.items():
                    if isinstance(subval, torch.Tensor):
                        shape_str = f"shape={tuple(subval.shape)}"
                        val_preview = subval.flatten()[:max_tensor_elements].tolist()
                        print(f"    - {subkey}: {shape_str}, dtype={subval.dtype}, values={val_preview}")
                    else:
                        print(f"    - {subkey}: {subval}")
        elif isinstance(value, torch.Tensor):
            print(f"➡️  Tensor | shape: {value.shape} | dtype: {value.dtype} | first values: {value.flatten()[:max_tensor_elements].tolist()}")
        else:
            print(f"➡️  {type(value)} | preview: {str(value)[:100]}")
        print()

Write the `MODEL_PATH`.

In [10]:
MODEL_PATH = "/home/ubuntu/Desktop/output_1/output_CcGAN_arch_SAGAN/saved_models/CcGAN_SAGAN_soft_nDsteps_2_checkpoint_intrain/CcGAN_checkpoint_niters_7500.pth"

Inspect.

In [11]:
inspect_pth_file(MODEL_PATH)

🔍 Inspecting file: /home/ubuntu/Desktop/output_1/output_CcGAN_arch_SAGAN/saved_models/CcGAN_SAGAN_soft_nDsteps_2_checkpoint_intrain/CcGAN_checkpoint_niters_7500.pth

📁 Top-level Keys and Types:
 - netG_state_dict: <class 'collections.OrderedDict'>
 - netD_state_dict: <class 'collections.OrderedDict'>
 - optimizerG_state_dict: <class 'dict'>
 - optimizerD_state_dict: <class 'dict'>
 - rng_state: <class 'torch.Tensor'>

=== [netG_state_dict] ===
➡️  Detected state_dict with 140 parameters.
  - module.snlinear0.bias                              | shape: (16384,) | dtype: torch.float32
  - module.snlinear0.weight_orig                       | shape: (16384, 128) | dtype: torch.float32
  - module.snlinear0.weight_u                          | shape: (16384,) | dtype: torch.float32
  - module.snlinear0.weight_v                          | shape: (128,) | dtype: torch.float32
  - module.block1.cond_bn1.bn.running_mean             | shape: (1024,) | dtype: torch.float32
  - module.block1.cond_bn1